####### Tasks ######

1. Look at the distribution of what's removed and what's added in terms of difference in old and new
2. Adding the repeated offenders back --> see if it works.
3. Removing outliers in terms of citations, collaborators, Altmetric score

In [14]:
import pandas as pd

In [2]:
df_new = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/RW_authors_w_confounders_filteredSample_postNHB.csv")

# df_new = df_new.rename(columns={'RetractedPaperMAGPID':'MAGPID',
#                                  'FieldName':'MAGFieldName',
#                                  'AcademicAge': 'AcademicAgeAtRetraction',
#                                  'AffRank': 'MAGRetractionYearAffRank',
#                                  'NumAuthorsInRetractedPaperRW':'NumAuthorsInRetractedPaper',
#                                  'cumPapers':'MAGCumPapersAtRetraction',
#                                  'cumCitations':'MAGCumCitationsAtRetraction',
#                                  'cumCollaborators':'MAGCumCollaboratorsAtRetraction',
#                                  'SJR':'SJRScoreRetractedPaperYear',
#                                  'SJR Best Quartile':'SJRQuartileRetractedPaperYear'})

df_old = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_effects_on_collaboration_networks/data/h4_altmetric/regression/RW_Authors_forRegression_rematching.csv")

df_new = df_new[df_new['nRetracted']==1]

/var/folders/bf/_6xjm2bn2_n17hfhskyq1pmw0000gp/T/ipykernel_94547/1567998168.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/RW_authors_w_confounders_filteredSample_postNHB.csv")


In [5]:
df_new[['MAGAID','RetractionYear']].drop_duplicates().to_csv("MAGAID_rYear_PostNHBData.csv", index=False)

In [13]:
df_old[['MAGAID','RetractionYear','AttritedClass']].drop_duplicates().to_csv("MAGAID_rYear_AttritedClass_PreNHBData.csv",
                                                                                        index=False)



In [11]:
df_old.columns.tolist()

['Record ID',
 'MAGPID',
 'RetractionYear',
 'OriginalPaperYear',
 'MAGAffIDRetractedPaper',
 'MAGRetractedPIDYear',
 'MAGAffRankRetractedPaper',
 'MAGFirstName',
 'GenderizeGender',
 'GenderizeConfidence',
 'FinalGender',
 'MAGFirstPubYear',
 'MAGCumPapersYearAtRetractionMinus1',
 'MAGCumPapersAtRetractionMinus1',
 'MAGCumCitationsYearAtRetractionMinus1',
 'MAGCumCitationsAtRetractionMinus1',
 'MAGCumCollaboratorsYearAtRetractionMinus1',
 'MAGCumCollaboratorsAtRetractionMinus1',
 'MAGFirstAffID',
 'MAGFirstYear',
 'MAGFirstAffiliationRank',
 'MAGAIDRankInRetractedPaper',
 'NumAuthorsInRetractedPaper',
 'MAGFirstAuthorFlag',
 'MAGLastAuthorFlag',
 'MAGRetractionYearAffID',
 'MAGRetractionYearAffYear',
 'MAGRetractionYearAffRank',
 'MAGAID',
 'MAGFieldID',
 'MAGFieldIDMaxPercent',
 'MAGFieldName',
 'Field_ART',
 'Field_BIOLOGY',
 'Field_BUSINESS',
 'Field_CHEMISTRY',
 'Field_COMPUTERSCIENCE',
 'Field_ECONOMICS',
 'Field_ENGINEERING',
 'Field_ENVIRONMENTALSCIENCE',
 'Field_GEOGRAPHY',
 '

In [284]:
df_new['MAGAID'].nunique(), df_new[df_new['RetractionYear'].le(2010)]['MAGAID'].nunique()

(14579, 5283)

In [195]:
magaids_common = set(df_new['MAGAID'].unique()).intersection(set(df_old['MAGAID'].unique()))
len(magaids_common)

13410

In [196]:
df_new_attritionClass = df_new[['MAGAID','AttritedClass']].drop_duplicates()
df_old_attritionClass = df_old[['MAGAID', 'AttritedClass']].drop_duplicates()

df_merged = df_old_attritionClass.merge(df_new_attritionClass, on='MAGAID')
magaids_notChanged = df_merged[df_merged['AttritedClass_x'] == df_merged['AttritedClass_y']]['MAGAID'].unique()


df_new_common = df_new[df_new['MAGAID'].isin(magaids_notChanged)]
df_old_common = df_old[df_old['MAGAID'].isin(magaids_notChanged)]
df_new_common['MAGAID'].nunique()

11388

In [197]:
df_old_common.to_csv('test.csv',index=False)

# Checking differences

In [106]:
def check_diff(dfold, dfnew, field):
    dfoldi = dfold[['MAGAID',field]].drop_duplicates()
    dfnewi = dfnew[['MAGAID',field]].drop_duplicates()
    
    dfmerged = dfoldi.merge(dfnewi, on=['MAGAID'])
    return dfmerged

In [107]:
# Checking differences in papers

df_diff = check_diff(df_old_common, df_new_common, 'MAGCumPapersAtRetraction')
df_diff['diffPapers'] = df_diff['MAGCumPapersAtRetraction_x']-df_diff['MAGCumPapersAtRetraction_y']
df_diff['diffPapers'].describe() # this is probably because of affiliations and retraction notices

count    11388.000000
mean         1.340710
std          5.097918
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max        138.000000
Name: diffPapers, dtype: float64

In [108]:
# Checking differences in citations

df_diff = check_diff(df_old_common, df_new_common, 'MAGCumCitationsAtRetraction')
df_diff['diffCites'] = df_diff['MAGCumCitationsAtRetraction_x']-df_diff['MAGCumCitationsAtRetraction_y']
df_diff['diffCites'].describe() # this is also probably because of affiliations and retraction notices

count    11388.000000
mean       390.723832
std       1686.005706
min          0.000000
25%          0.000000
50%          8.000000
75%        136.000000
max      51650.000000
Name: diffCites, dtype: float64

In [109]:
# checking differences in collaborators

df_diff = check_diff(df_old_common, df_new_common, 'MAGCumCollaboratorsAtRetraction')
df_diff['diffCollabs'] = df_diff['MAGCumCollaboratorsAtRetraction_x']-df_diff['MAGCumCollaboratorsAtRetraction_y']
df_diff['diffCollabs'].describe() # This also makes sense

count    11388.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: diffCollabs, dtype: float64

In [110]:
# Checking difference in academic age

df_diff = check_diff(df_old_common, df_new_common, 'AcademicAgeAtRetraction')
df_diff['diffAge'] = df_diff['AcademicAgeAtRetraction_x']-df_diff['AcademicAgeAtRetraction_y']
df_diff['diffAge'].describe() # This also makes sense

count    11388.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: diffAge, dtype: float64

In [111]:
# Checking difference in gender

df_diff = check_diff(df_old_common, df_new_common, 'GenderizeGender')
df_diff[df_diff['GenderizeGender_x'] != df_diff['GenderizeGender_y']] # this would have been very weird if different

,MAGAID,GenderizeGender_x,GenderizeGender_y


In [112]:
# Checking for retraction year
df_diff = check_diff(df_old_common, df_new_common, 'RetractionYear')
df_diff['diffRyear'] = df_diff['RetractionYear_x']-df_diff['RetractionYear_y']
df_diff['diffRyear'].describe() # This also makes sense

count    11388.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: diffRyear, dtype: float64

In [113]:
# Checking retraction reason
df_diff = check_diff(df_old_common, df_new_common, 'ReasonPropagatedMajorityOfMajority')
df_diff[df_diff['ReasonPropagatedMajorityOfMajority_x'] != df_diff['ReasonPropagatedMajorityOfMajority_y']]

,MAGAID,ReasonPropagatedMajorityOfMajority_x,ReasonPropagatedMajorityOfMajority_y
1461,6208356.0,misconduct,plagiarism


In [114]:
# Checking record ID
df_diff = check_diff(df_old_common, df_new_common, 'Record ID')
df_diff[df_diff['Record ID_x'] != df_diff['Record ID_y']]

,MAGAID,Record ID_x,Record ID_y
1461,6208356.0,656,1410


In [115]:
# Checking affiliation ranking
df_diff = check_diff(df_old_common, df_new_common, 'MAGRetractionYearAffRankOrdinal')
df_diff[df_diff['MAGRetractionYearAffRankOrdinal_x'] != df_diff['MAGRetractionYearAffRankOrdinal_y']]

,MAGAID,MAGRetractionYearAffRankOrdinal_x,MAGRetractionYearAffRankOrdinal_y


In [116]:
# Checking for author order
df_diff = check_diff(df_old_common, df_new_common, 'MAGAIDRankTypeInRetractedPaper')
df_diff[df_diff['MAGAIDRankTypeInRetractedPaper_x'] != df_diff['MAGAIDRankTypeInRetractedPaper_y']]

pd.crosstab(df_diff['MAGAIDRankTypeInRetractedPaper_x'], df_diff['MAGAIDRankTypeInRetractedPaper_y'])
# This is because NumAuthorsInRetractedPaper has changed due to change in logic

MAGAIDRankTypeInRetractedPaper_y,First or Last or Only Author,Middle Author
MAGAIDRankTypeInRetractedPaper_x,,
First or Last or Only Author,4460,80
Middle Author,67,6781


In [117]:
# Checking NumAuthorsInRetractedPaper
df_diff = check_diff(df_old_common, df_new_common, 'NumAuthorsInRetractedPaper')
df_diff[df_diff['NumAuthorsInRetractedPaper_x'] != df_diff['NumAuthorsInRetractedPaper_y']]

,MAGAID,NumAuthorsInRetractedPaper_x,NumAuthorsInRetractedPaper_y
148,2.404170e+09,4.0,3.0
178,2.304832e+09,15.0,14.0
179,2.915638e+09,15.0,14.0
204,2.147850e+09,10.0,9.0
379,3.592250e+07,4.0,3.0
...,...,...,...
11175,2.293868e+09,4.0,5.0
11176,3.089276e+09,4.0,5.0
11222,2.114455e+09,7.0,6.0
11282,2.806438e+09,18.0,19.0


In [101]:
(df_diff['NumAuthorsInRetractedPaper_x']-df_diff['NumAuthorsInRetractedPaper_y']).describe()

count    11388.000000
mean         0.102388
std          0.996905
min         -7.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         18.000000
dtype: float64

In [75]:
# Checking Journal Type
df_diff = check_diff(df_old_common, df_new_common, 'MAGJournalType')
df_diff[df_diff['MAGJournalType_x'] != df_diff['MAGJournalType_y']] # need to check this

,MAGAID,MAGJournalType_x,MAGJournalType_y
940,2.305203e+09,journal,conference
1094,4.732730e+07,journal,conference
1095,2.015425e+09,journal,conference
1096,2.062818e+09,journal,conference
1097,2.122246e+09,journal,conference
1098,2.130523e+09,journal,conference
1099,2.304089e+09,journal,conference
1100,2.922743e+09,journal,conference
2415,2.090772e+09,journal,NaN
2543,2.143278e+09,journal,conference


In [76]:
# Checking Journal Type
# need to explore this
df_diff = check_diff(df_old_common, df_new_common, 'SJRQuartileRetractedPaperYear')
df_diff = df_diff[~df_diff['SJRQuartileRetractedPaperYear_x'].isna() & ~df_diff['SJRQuartileRetractedPaperYear_y'].isna()]
df_diff[df_diff['SJRQuartileRetractedPaperYear_x'] != df_diff['SJRQuartileRetractedPaperYear_y']] # need to check this

,MAGAID,SJRQuartileRetractedPaperYear_x,SJRQuartileRetractedPaperYear_y
3939,1.208293e+09,2.0,3.0
3940,2.016158e+09,2.0,3.0
3941,2.149865e+09,2.0,3.0
4074,1.212782e+09,1.0,4.0
4075,2.345302e+09,1.0,4.0
4265,3.093544e+09,2.0,3.0
4324,2.121659e+09,1.0,4.0
5044,2.314296e+09,1.0,2.0
5922,2.623592e+09,3.0,2.0
6286,3.067086e+09,2.0,1.0


In [88]:
# checking fields

# Lowercase columns with 'Field_' prefix
for col in df_old_common.columns:
    if col.startswith('Field_'):
        df_old_common.rename(columns={col: col.lower()}, inplace=True)

for col in df_new_common.columns:
    if col.startswith('Field'):
        df_new_common.rename(columns={col: col.lower()}, inplace=True)


# Iterate through lowercase columns with 'Field_' prefix
for column_name in df_old_common.columns:
    if column_name.startswith('field_'):
        df_diff = check_diff(df_old_common, df_new_common, column_name)
        diff_mask = df_diff[column_name+'_x'] != df_diff[column_name+'_y']
        print(df_diff[diff_mask])


Empty DataFrame
Columns: [MAGAID, field_art_x, field_art_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_biology_x, field_biology_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_business_x, field_business_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_chemistry_x, field_chemistry_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_computerscience_x, field_computerscience_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_economics_x, field_economics_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_engineering_x, field_engineering_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_environmentalscience_x, field_environmentalscience_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_geography_x, field_geography_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_geology_x, field_geology_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_history_x, field_history_y]
Index: []
Empty DataFrame
Columns: [MAGAID, field_materialsscience_x, field_materialsscience_y]


# Exploring repeated offenders

In [270]:
fieldid_to_fieldname = {
        "age": ["AcademicAgeAtRetraction"],
        "gender": ["GenderizeGender"],
        "yearofretraction": ["RetractionYear"],
        "logcitations":["LogMAGCumCitationsAtRetraction"],
        "logcollaborators":["LogMAGCumCollaboratorsAtRetraction"],
        "papers":["MAGCumPapersAtRetraction"],
        "reasons":["ReasonPropagatedMajorityOfMajority"],
        "affrank":["MAGRetractionYearAffRankOrdinal"],
        "contributionrank":["MAGAIDRankTypeInRetractedPaper"],
        "numauthors":["NumAuthorsInRetractedPaper"],
        "venue":["MAGJournalType"],
        "impactfactor":["SJRQuartileRetractedPaperYear"],
        "field": list(df_new.filter(regex=("Field_.*")).columns)
}

relevant_cols = ['Record ID','MAGAID','MAGPID','MAGAuthorOrder', 'MAGAuthorName','MAGCumCitationsAtRetraction',
                     'MAGCumCollaboratorsAtRetraction','MAGRetractionYearAffRank', 'AltmetricScoreAtRetraction']

for val in fieldid_to_fieldname.values():
    for vali in val:
        relevant_cols.append(vali)

relevant_cols

['Record ID',
 'MAGAID',
 'MAGPID',
 'MAGAuthorOrder',
 'MAGAuthorName',
 'MAGCumCitationsAtRetraction',
 'MAGCumCollaboratorsAtRetraction',
 'MAGRetractionYearAffRank',
 'AltmetricScoreAtRetraction',
 'AcademicAgeAtRetraction',
 'GenderizeGender',
 'RetractionYear',
 'LogMAGCumCitationsAtRetraction',
 'LogMAGCumCollaboratorsAtRetraction',
 'MAGCumPapersAtRetraction',
 'ReasonPropagatedMajorityOfMajority',
 'MAGRetractionYearAffRankOrdinal',
 'MAGAIDRankTypeInRetractedPaper',
 'NumAuthorsInRetractedPaper',
 'MAGJournalType',
 'SJRQuartileRetractedPaperYear',
 'Field_art',
 'Field_biology',
 'Field_business',
 'Field_chemistry',
 'Field_computerscience',
 'Field_economics',
 'Field_engineering',
 'Field_environmentalscience',
 'Field_geography',
 'Field_geology',
 'Field_history',
 'Field_materialsscience',
 'Field_mathematics',
 'Field_medicine',
 'Field_philosophy',
 'Field_physics',
 'Field_politicalscience',
 'Field_psychology',
 'Field_sociology']

In [103]:
# checking repeated offenders



df_new = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/RW_authors_w_confounders_filteredSample_postNHB.csv")
df_repeated = df_new[df_new['nRetracted'] > 1].drop_duplicates()[relevant_cols].drop_duplicates()

/var/folders/bf/_6xjm2bn2_n17hfhskyq1pmw0000gp/T/ipykernel_94547/2637139170.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/RW_authors_w_confounders_filteredSample_postNHB.csv")


NameError: name 'relevant_cols' is not defined

In [18]:
# Importing relevant packages

import pandas as pd
import os
from config_reader import read_config
from rapidfuzz import process, fuzz
# Reading paths
paths = read_config()

RW_ORIGINAL_W_YEAR_LOCAL_PATH = paths['RW_ORIGINAL_W_YEAR_LOCAL_PATH']
df_rw = pd.read_csv(RW_ORIGINAL_W_YEAR_LOCAL_PATH, usecols=['RetractionDate','Record ID','Author'])\
            .drop_duplicates()

In [174]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df_repeated.merge(df_rw, on='Record ID').sort_values(by=['MAGAID','RetractionDate'])\
        .drop(columns=['RetractionDate','MAGAIDRankTypeInRetractedPaper','MAGAuthorOrder',
                      'ReasonPropagatedMajorityOfMajority','MAGPID','Record ID',
                      'MAGJournalType','SJRQuartileRetractedPaperYear','Author',
                      'NumAuthorsInRetractedPaper']).drop_duplicates()

,MAGAID,MAGAuthorName,AcademicAgeAtRetraction,GenderizeGender,RetractionYear,LogMAGCumCitationsAtRetraction,LogMAGCumCollaboratorsAtRetraction,MAGCumPapersAtRetraction,MAGRetractionYearAffRankOrdinal,Field_art,Field_biology,Field_business,Field_chemistry,Field_computerscience,Field_economics,Field_engineering,Field_environmentalscience,Field_geography,Field_geology,Field_history,Field_materialsscience,Field_mathematics,Field_medicine,Field_philosophy,Field_physics,Field_politicalscience,Field_psychology,Field_sociology
810,11607975,daniel f klessig,29.0,male,2004.0,8.496990,5.541264,189,13.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2537,27070073,kalimuthu kalishwaralal,2.0,male,2010.0,4.330733,3.555348,23,1500.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
874,29230377,vera schramke,2.0,female,2005.0,5.703782,2.484907,5,31.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1965,33510457,gaurav vadnerkar,1.0,male,2008.0,1.098612,2.302585,5,1500.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3446,38001864,yannis georgalis,0.0,male,2013.0,0.000000,1.386294,2,57.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691,3174584132,david l brown,43.0,male,2007.0,6.755769,5.111988,98,20.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2481,3174785825,jae ho lee,0.0,male,2009.0,1.386294,2.564949,1,1500.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2567,3175629790,eddy s leman,5.0,male,2012.0,5.303305,3.367296,10,1500.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1788,3175900097,zhiguo wang,4.0,male,2011.0,4.043051,3.761200,5,175.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [176]:
df_repeated.merge(df_rw, on='Record ID').sort_values(by=['MAGAID','RetractionDate'])\
            .drop_duplicates(subset=['MAGAID','MAGAuthorOrder','NumAuthorsInRetractedPaper',
                                    'MAGAIDRankTypeInRetractedPaper'])

,Record ID,MAGAID,MAGPID,MAGAuthorOrder,MAGAuthorName,AcademicAgeAtRetraction,GenderizeGender,RetractionYear,LogMAGCumCitationsAtRetraction,LogMAGCumCollaboratorsAtRetraction,MAGCumPapersAtRetraction,ReasonPropagatedMajorityOfMajority,MAGRetractionYearAffRankOrdinal,MAGAIDRankTypeInRetractedPaper,NumAuthorsInRetractedPaper,MAGJournalType,SJRQuartileRetractedPaperYear,Field_art,Field_biology,Field_business,Field_chemistry,Field_computerscience,Field_economics,Field_engineering,Field_environmentalscience,Field_geography,Field_geology,Field_history,Field_materialsscience,Field_mathematics,Field_medicine,Field_philosophy,Field_physics,Field_politicalscience,Field_psychology,Field_sociology,Author,RetractionDate
810,203,11607975,2006124330,4,daniel f klessig,29.0,male,2004.0,8.496990,5.541264,189,mistake,13.0,First or Last or Only Author,4.0,journal,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Meena R Chandok;A Jimmy Ytterberg;Klaas J van Wijk;Daniel F Klessig,2004-10-29
2537,1236,27070073,2101809587,1,kalimuthu kalishwaralal,2.0,male,2010.0,4.330733,3.555348,23,misconduct,1500.0,First or Last or Only Author,4.0,journal,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ramanathan Vaidyanathan;Kalimuthu Kalishwaralal;Shubaash Gopalram;Sangiliyandi Gurunathan,2010-11-01
2538,1236,27070073,2101809587,2,kalimuthu kalishwaralal,2.0,male,2010.0,4.330733,3.555348,23,misconduct,1500.0,Middle Author,4.0,journal,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ramanathan Vaidyanathan;Kalimuthu Kalishwaralal;Shubaash Gopalram;Sangiliyandi Gurunathan,2010-11-01
874,979,29230377,2070987160,1,vera schramke,2.0,female,2005.0,5.703782,2.484907,5,mistake,31.0,First or Last or Only Author,2.0,journal,1.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Vera Schramke;Robin Allshire,2005-10-07
1403,2222,29230377,2031586574,1,vera schramke,2.0,female,2005.0,5.703782,2.484907,5,mistake,31.0,First or Last or Only Author,7.0,journal,1.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Vera Schramke;Daniel M Sheedy;Ahmet M Denli;Carolina Bonila;Karl Ekwall;Gregory J Hannon;Robin C Allshire,2005-10-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691,2145,3174584132,2080060897,1,david l brown,43.0,male,2007.0,6.755769,5.111988,98,mistake,20.0,First or Last or Only Author,7.0,journal,2.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,David L Brown;Peter J Meagher;Kenneth R Knight;Effie Keramidaris;Rosalind Romeo-Meeuw;Anthony J Penington;Wayne A Morrison,2007-10-07
2481,861,3174785825,2044613360,2,jae ho lee,0.0,male,2009.0,1.386294,2.564949,1,plagiarism,1500.0,Middle Author,13.0,journal,1.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Jae Ho Lee;Majlinda Lako;Lyle Armstrong;Mary Herbert;Manyu Li;Wolfgang Engel;Xin Zhang;David Elliott;Miodrag Stojkovic;John Parrington;Alison Murdoch;Tom Strachan;Karim Nayernia,2009-09-01
2567,2629,3175629790,1982308453,1,eddy s leman,5.0,male,2012.0,5.303305,3.367296,10,mistake,1500.0,First or Last or Only Author,6.0,journal,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Eddy S Leman;Ahmed Magheli;Grant W Cannon;Leslie Mangold;Alan W Partin;Robert H Getzenberg,2012-07-01
1788,4604,3175900097,2037997249,7,zhiguo wang,4.0,male,2011.0,4.043051,3.761200,5,misconduct,175.0,First or Last or Only Author,7.0,journal,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Xiaobin Luo;Jiening Xiao;Huixian Lin;Baoxin Li;Yanjie Lu;Baofeng Yang;Zhiguo Wang,2011-11-23


In [177]:
AUTHORS_SINGLE_AND_REPEATED_OFFENDERS_PATH = paths['AUTHORS_SINGLE_AND_REPEATED_OFFENDERS_PATH']
dftemp = pd.read_csv(AUTHORS_SINGLE_AND_REPEATED_OFFENDERS_PATH)

In [193]:
df2 = dftemp[dftemp.nRetracted.gt(1)].merge(df_rw, on=['Record ID'])\
        .sort_values(by='RetractionDate')

df3 = df2.groupby('MAGAID')['RetractionDate'].min().reset_index().rename(columns={'RetractionDate':'FirstRetractionDate'})

df4 = df2.merge(df3, on='MAGAID')

df5 = df4[df4['RetractionDate'] == df4['FirstRetractionDate']]

df5[df5.duplicated(subset='MAGAID', keep=False)]


,MAGAID,MAGAuthorName,Record ID,RetractedPaperMAGPID,RetractionYear,nRetracted,Author,RetractionDate,FirstRetractionDate
7,2689068837,j goshima,3021,2411045189,1991.0,2,Jun Goshima,1991-09-01,1991-09-01
8,2689068837,j goshima,3020,2411045189,1991.0,2,Jun Goshima,1991-09-01,1991-09-01
31,2296168294,seema dhir,17319,2034702085,1993.0,2,Sarwan K Dhir;Seema Dhir;Michael A Savka;Faith Belanger;Alan L Kriz;Stephen K Farrand;JacK M Widholm,1993-05-01,1993-05-01
32,2296168294,seema dhir,18602,2026915809,1993.0,2,Sarwan K Dhir;Seema Dhir;Ann Pizanis Sturtevant;Jack M Widholm,1993-05-01,1993-05-01
33,1824918633,sarwan k dhir,17319,2034702085,1993.0,2,Sarwan K Dhir;Seema Dhir;Michael A Savka;Faith Belanger;Alan L Kriz;Stephen K Farrand;JacK M Widholm,1993-05-01,1993-05-01
...,...,...,...,...,...,...,...,...,...
3110,1433263031,amir reza jalilian,379,2165474867,2015.0,2,Hassan Yousefnia;Amir Reza Jalilian;Samaneh Zolghadri,2015-12-18,2015-12-18
3111,2041607459,hassan yousefnia,380,2083559333,2015.0,2,Samaneh Zolghadri;Amir Reza Jalilian;Hassan Yousefnia;Ali Bahrami-Samani;Mohammad Ghannadi-Maragheh,2015-12-18,2015-12-18
3112,2041607459,hassan yousefnia,379,2165474867,2015.0,2,Hassan Yousefnia;Amir Reza Jalilian;Samaneh Zolghadri,2015-12-18,2015-12-18
3113,2466543429,samaneh zolghadri,380,2083559333,2015.0,2,Samaneh Zolghadri;Amir Reza Jalilian;Hassan Yousefnia;Ali Bahrami-Samani;Mohammad Ghannadi-Maragheh,2015-12-18,2015-12-18


# Exploring distribution of those that are not in the intersection

Note that intersection here means (i) authors that occur in both new and old AND (ii) who have the same position in terms of attrited vs. non-attrited

In [277]:
fieldid_to_fieldname = {
        "age": ["AcademicAgeAtRetraction"],
        "gender": ["GenderizeGender"],
        "yearofretraction": ["RetractionYear"],
        "logcitations":["LogMAGCumCitationsAtRetraction"],
        "logcollaborators":["LogMAGCumCollaboratorsAtRetraction"],
        "papers":["MAGCumPapersAtRetraction"],
        "reasons":["ReasonPropagatedMajorityOfMajority"],
        "affrank":["MAGRetractionYearAffRankOrdinal"],
        "contributionrank":["MAGAIDRankTypeInRetractedPaper"],
        "numauthors":["NumAuthorsInRetractedPaper"],
        "venue":["MAGJournalType"],
        "impactfactor":["SJRQuartileRetractedPaperYear"],
        "field": list(df_old.filter(regex=("Field_.*")).columns)
}
relevant_cols_old = ['Record ID','MAGAID','MAGPID','MAGCumCitationsAtRetraction',
                     'MAGCumCollaboratorsAtRetraction','MAGRetractionYearAffRank',
                    'AltmetricScoreAtRetraction']

for val in fieldid_to_fieldname.values():
    for vali in val:
        relevant_cols_old.append(vali)


In [278]:
dfnew_diff = df_new[~df_new['MAGAID'].isin(magaids_notChanged)][relevant_cols].drop_duplicates()
dfold_diff = df_old[~df_old['MAGAID'].isin(magaids_notChanged)][relevant_cols_old].drop_duplicates()

dfnew_diff['MAGAID'].nunique(), dfold_diff['MAGAID'].nunique()

(3191, 4226)

In [273]:
# Check difference continuous
def check_diff_cont(field):
    print(field)
    dfoldi = dfold_diff[['MAGAID',field]].drop_duplicates()
    dfnewi = dfnew_diff[['MAGAID',field]].drop_duplicates()
    display(dfoldi[field].describe(), dfnewi[field].describe())

# Check difference nominal
def check_diff_nom(field):
    print(field)
    dfoldi = dfold_diff[['MAGAID', field]].drop_duplicates()
    dfnewi = dfnew_diff[['MAGAID', field]].drop_duplicates()
    
    total_old = len(dfoldi)
    total_new = len(dfnewi)
    
    old_percentages = (dfoldi[field].value_counts() / total_old) * 100
    new_percentages = (dfnewi[field].value_counts() / total_new) * 100
    
    display(old_percentages, new_percentages)

from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [255]:
check_diff_cont('AcademicAgeAtRetraction')

AcademicAgeAtRetraction


count    4226.000000
mean       13.925698
std        12.901527
min         0.000000
25%         3.000000
50%        10.000000
75%        21.000000
max        63.000000
Name: AcademicAgeAtRetraction, dtype: float64

count    4593.000000
mean       15.310908
std        12.759962
min         0.000000
25%         5.000000
50%        12.000000
75%        23.000000
max        68.000000
Name: AcademicAgeAtRetraction, dtype: float64

In [256]:
check_diff_cont('MAGCumPapersAtRetraction')

MAGCumPapersAtRetraction


count    4226.000000
mean       47.872220
std        99.925277
min         1.000000
25%         3.000000
50%        12.000000
75%        47.000000
max      1247.000000
Name: MAGCumPapersAtRetraction, dtype: float64

count    4593.000000
mean       56.703897
std       103.077391
min         1.000000
25%         5.000000
50%        18.000000
75%        62.000000
max      1313.000000
Name: MAGCumPapersAtRetraction, dtype: float64

In [257]:
check_diff_cont('MAGCumCitationsAtRetraction')

MAGCumCitationsAtRetraction


count     4226.000000
mean       985.043303
std       3171.344962
min          0.000000
25%         11.000000
50%         86.500000
75%        545.250000
max      52517.000000
Name: MAGCumCitationsAtRetraction, dtype: float64

count     4593.000000
mean       998.190943
std       3050.418021
min          0.000000
25%         20.000000
50%        148.000000
75%        681.000000
max      67023.000000
Name: MAGCumCitationsAtRetraction, dtype: float64

In [258]:
check_diff_cont('MAGCumCollaboratorsAtRetraction')

MAGCumCollaboratorsAtRetraction


count    4226.000000
mean      133.781354
std       382.662824
min         0.000000
25%         9.000000
50%        29.000000
75%       101.750000
max      7896.000000
Name: MAGCumCollaboratorsAtRetraction, dtype: float64

count     4593.000000
mean       162.447420
std        531.925443
min          0.000000
25%         12.000000
50%         41.000000
75%        131.000000
max      15764.000000
Name: MAGCumCollaboratorsAtRetraction, dtype: float64

In [259]:
check_diff_cont('RetractionYear')

RetractionYear


count    4226.000000
mean     2011.122575
std         4.164843
min      1990.000000
25%      2010.000000
50%      2012.000000
75%      2014.000000
max      2015.000000
Name: RetractionYear, dtype: float64

count    4593.000000
mean     2010.794470
std         4.400912
min      1990.000000
25%      2009.000000
50%      2012.000000
75%      2014.000000
max      2015.000000
Name: RetractionYear, dtype: float64

In [264]:
check_diff_cont('NumAuthorsInRetractedPaper')

NumAuthorsInRetractedPaper


count    4226.000000
mean        6.942499
std         4.663722
min         1.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        34.000000
Name: NumAuthorsInRetractedPaper, dtype: float64

count    5404.000000
mean        6.778682
std         4.194717
min         1.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        33.000000
Name: NumAuthorsInRetractedPaper, dtype: float64

In [261]:
check_diff_nom('ReasonPropagatedMajorityOfMajority')

ReasonPropagatedMajorityOfMajority


ReasonPropagatedMajorityOfMajority
plagiarism    32.087080
mistake       25.769049
misconduct    24.680549
other         17.463322
Name: count, dtype: float64

ReasonPropagatedMajorityOfMajority
plagiarism    29.926260
misconduct    28.594838
mistake       26.526014
other         14.563703
Name: count, dtype: float64

In [262]:
check_diff_nom('GenderizeGender')

GenderizeGender


GenderizeGender
male      74.325603
female    25.674397
Name: count, dtype: float64

GenderizeGender
male      74.548226
female    25.451774
Name: count, dtype: float64

In [263]:
check_diff_nom('MAGRetractionYearAffRank')

MAGRetractionYearAffRank


MAGRetractionYearAffRank
1001-      35.284934
201-300     8.157689
101-150     6.693989
151-200     6.537861
301-400     5.386417
             ...    
58          0.039032
79          0.039032
37          0.019516
87          0.019516
48          0.019516
Name: count, Length: 86, dtype: float64

MAGRetractionYearAffRank
1001-      36.407421
201-300     7.438459
101-150     6.475205
301-400     6.047092
151-200     5.601142
             ...    
43          0.053514
62          0.053514
46          0.053514
63          0.035676
19          0.035676
Name: count, Length: 86, dtype: float64

In [265]:
check_diff_nom('MAGAIDRankTypeInRetractedPaper')

MAGAIDRankTypeInRetractedPaper


MAGAIDRankTypeInRetractedPaper
Middle Author                   60.67203
First or Last or Only Author    39.32797
Name: count, dtype: float64

MAGAIDRankTypeInRetractedPaper
Middle Author                   60.112247
First or Last or Only Author    39.887753
Name: count, dtype: float64

In [266]:
check_diff_nom('MAGJournalType')

MAGJournalType


MAGJournalType
journal       97.988642
conference     2.011358
Name: count, dtype: float64

MAGJournalType
journal       91.408115
conference     0.846171
Name: count, dtype: float64

In [267]:
check_diff_nom('SJRQuartileRetractedPaperYear')

SJRQuartileRetractedPaperYear


SJRQuartileRetractedPaperYear
1.0    51.325130
2.0    15.499290
3.0     5.750118
4.0     0.709891
Name: count, dtype: float64

SJRQuartileRetractedPaperYear
1.0    50.738007
2.0    12.136121
3.0     4.530545
4.0     0.779008
Name: count, dtype: float64

In [274]:
check_diff_cont('AltmetricScoreAtRetraction')

AltmetricScoreAtRetraction


count    4226.000000
mean        5.565606
std        61.503455
min         0.000000
25%         0.000000
50%         0.000000
75%         5.000000
max      3756.500000
Name: AltmetricScoreAtRetraction, dtype: float64

count    5136.000000
mean       10.237442
std       108.815051
min         0.000000
25%         0.000000
50%         0.000000
75%         6.000000
max      3756.500000
Name: AltmetricScoreAtRetraction, dtype: float64

## Exploring Year of Attrition Thresholds

In [131]:
from config_reader import read_config

# Reading paths
paths = read_config()
# AUTHORS_W_CONFOUNDERS_PATH = paths['AUTHORS_W_CONFOUNDERS_PATH']
# RW_ORIGINAL_W_YEAR_LOCAL_PATH = paths['RW_ORIGINAL_W_YEAR_LOCAL_PATH']
# SCIMAGO_PATH_LOCAL = paths['SCIMAGO_PATH_LOCAL']
# MAG_PATH_LOCAL = paths['MAG_PATH_LOCAL']
# MAG_FIELD_NAMES_LOCAL = paths['MAG_FIELD_NAMES_LOCAL']
# Path to where we will save our processed files
OUTDIR = paths['PROCESSED_FOLDER_LOCAL']

df_authors = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/RW_authors_w_confounders_filteredSample_postNHB.csv")
df_authors = df_authors.rename(columns={'AttritedClass':'AttritedClass6'})\
                .drop(columns=['YearOfAttrition','AttritedClassRobust','YearsActive','OriginalPaperYear'])



/var/folders/bf/_6xjm2bn2_n17hfhskyq1pmw0000gp/T/ipykernel_94547/1338723568.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_authors = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/RW_authors_w_confounders_filteredSample_postNHB.csv")


In [132]:
# Reading RW to extract attrition date for those with NaN
df_rw = pd.read_csv(RW_ORIGINAL_W_YEAR_LOCAL_PATH,
                   usecols=['Record ID','OriginalPaperDate'])


df_rw['OriginalPaperDate'] = pd.to_datetime(df_rw['OriginalPaperDate'])

# Extracting the year
df_rw['OriginalPaperYear'] = df_rw['OriginalPaperDate'].dt.year

In [133]:
# Let us add attrition flags
# Let us first read Bedoor's file
df_attrition_year_Bedoor = pd.read_csv(OUTDIR+"AIDs_YearOfAttrition_95percentile.csv")\
                            .rename(columns={'AID':'MAGAID'})

df_attrition_year_Bedoor

,MAGAID,YearOfAttrition
0,859,1940.0
1,7539,1938.0
2,11523,1975.0
3,19669,1993.0
4,26851,1964.0
...,...,...
48672004,3177495302,2015.0
48672005,3177495427,-1.0
48672006,3177497064,2015.0
48672007,3177497129,2015.0


In [134]:
df_attrition_year_Bedoor = df_attrition_year_Bedoor[df_attrition_year_Bedoor['MAGAID'].isin(df_authors['MAGAID'])]

# Merging with df_authors
df_authors = df_authors.merge(df_attrition_year_Bedoor, on='MAGAID', how='left')

# Separating those with year of attrition from those without
df_authors_w_yearOfAttrition = df_authors[~df_authors['YearOfAttrition'].isna()]

# Processing those without
df_authors_wo_yearOfAttrition = df_authors[df_authors['YearOfAttrition'].isna()]
df_authors_wo_yearOfAttrition = df_authors_wo_yearOfAttrition.merge(df_rw, on='Record ID', how='left')
df_authors_wo_yearOfAttrition['YearOfAttrition'] = df_authors_wo_yearOfAttrition['OriginalPaperYear']

df_authors = pd.concat([df_authors_w_yearOfAttrition,df_authors_wo_yearOfAttrition])

def compute_years_active(row):
    if row['YearOfAttrition']  == -1:
        return None
    else:
        return row['YearOfAttrition']-row['RetractionYear']

# Now computing the years active 
df_authors['YearsActive'] = df_authors.apply(lambda row: compute_years_active(row), axis=1)

df_authors = df_authors.drop(columns=['OriginalPaperYear', 'OriginalPaperDate'])
df_authors = df_authors.merge(df_rw, on='Record ID', how='left')

In [135]:
# Extract attrition class
def extract_attrited_class(row, lowerthreshold=-1, upperthreshold=0):
    if(row['YearsActive'] > upperthreshold):
        # That means it is not attrited author
        return 0
    elif(row['YearsActive'] in list(range(lowerthreshold,upperthreshold+1))):
        # That means it is attrited
        return 1
    elif pd.isnull(row['YearsActive']):
        return 0
    else:
        # That means it's attrited but not due to retraction 
        return -1

# Flagging Attrited vs Non-Attrited authors

df_authors['AttritedClass'] = df_authors.apply(lambda row: extract_attrited_class(row), axis=1)

df_authors['AttritedClassRobust'] = df_authors.apply(lambda row: extract_attrited_class(row,
                                                                            upperthreshold=1), 
                                                                             axis=1)

In [136]:
# We are going to remove authors that were changed in terms of their year of attrition due to retraction notices

# Read PubHistories file
histories = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/Retracted_Authors_PubHistories.csv")
# Read notices file
notices = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/retraction_notices_postfiltering.csv")
# Removing notices from histories

histories.head()


,MAGPID,MAGAID,MAGAffID,MAGAuthorOrder,MAGTitle,MAGPubYear,MAGAuthorName,Record ID,RetractedPaperMAGPID,RetractionYear
0,524,569306176,148283060.0,1,the cryo thermochromatographic separator cts a...,2002.0,u w kirbach,23311,2273790572,2002.0
1,1598005161,569306176,148283060.0,8,non observation of the production of superheav...,2002.0,u w kirbach,23311,2273790572,2002.0
2,1968637420,569306176,NaN,5,publisher s note confirmation of production of...,2003.0,u w kirbach,23311,2273790572,2002.0
3,1989365649,569306176,148283060.0,14,chemical investigation of hassium element 108,2002.0,u w kirbach,23311,2273790572,2002.0
4,2006519711,569306176,148283060.0,16,chemical and nuclear studies of hassium and el...,2004.0,u w kirbach,23311,2273790572,2002.0


In [139]:
papers_before_retraction = histories[histories['MAGPubYear'].le(histories['RetractionYear'])]\
                            .groupby('MAGAID')['MAGPID'].nunique().reset_index()\
                            .rename(columns={'MAGPID':'numPapers'})

papers_before_retraction_noNotice = histories[histories['MAGPubYear'].le(histories['RetractionYear']) & 
                                             ~histories['MAGPID'].isin(notices['PID'])]\
                            .groupby('MAGAID')['MAGPID'].nunique().reset_index()\
                            .rename(columns={'MAGPID':'numPapersNoNotice'})

merged = papers_before_retraction.merge(papers_before_retraction_noNotice, how='left')
merged

,MAGAID,numPapers,numPapersNoNotice
0,185294,19,19
1,224730,11,11
2,419208,45,44
3,733318,118,117
4,861018,59,59
...,...,...,...
23615,3177295229,1,1
23616,3177297137,1,1
23617,3177442959,2,1
23618,3177468043,5,5


In [140]:
merged['diff'] = merged['numPapers']-merged['numPapersNoNotice']
merged['diff'].describe()

count    23620.000000
mean         0.276969
std          0.652439
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         24.000000
Name: diff, dtype: float64

In [141]:
authors_wrongly_identified = merged[merged['diff'].gt(0)]

df_authors = df_authors[~df_authors['MAGAID'].isin(authors_wrongly_identified['MAGAID'])]

In [142]:
df_authors[df_authors['AttritedClass'] == 0][['RetractionYear','YearOfAttrition', 'YearsActive']].drop_duplicates()\
                .sort_values(by='YearsActive')

,RetractionYear,YearOfAttrition,YearsActive
1156,2001.0,2002.0,1.0
500,1998.0,1999.0,1.0
475,1999.0,2000.0,1.0
472,1992.0,1993.0,1.0
9261,2013.0,2014.0,1.0
...,...,...,...
909,2002.0,-1.0,NaN
929,2010.0,-1.0,NaN
975,2014.0,-1.0,NaN
1077,2011.0,-1.0,NaN


In [143]:
dftemp = df_authors[['MAGAID','AttritedClass','AttritedClass6']].drop_duplicates()

pd.crosstab(dftemp['AttritedClass6'],dftemp['AttritedClass'])

AttritedClass,-1,0,1
AttritedClass6,,,
-1,948,1335,28
0,8,7568,1
1,0,325,1856


In [144]:
df_authors[df_authors['YearOfAttrition'].isna()]

,MAGAID,MAGAuthorName,Record ID,MAGPID,RetractionYear,nRetracted,dateobject,JID,CSID,JournalName,...,AltmetricScorePreRetraction,AltmetricScorePostRetraction,YearOfAttrition,OriginalPaperDate_x,OriginalPaperDate_y,YearsActive,OriginalPaperDate,OriginalPaperYear,AttritedClass,AttritedClassRobust


In [145]:
df_authors.to_csv("test_gap_95percent.csv",index=False)

In [146]:
df_authors['MAGAID'].nunique()

12069

In [365]:
danger=True
def get_oldattritionclass(row):
    if row['YearsActive'] in [0,-1]:
        return 1
    elif row['YearsActive'] > 0:
        return 0
    else:
        return -1

if(danger):
    df_pubhistories = pd.read_csv("../../data/processed/Retracted_Authors_PubHistories.csv",
                                 usecols=['MAGAID','RetractionYear','MAGPubYear'])\
                            .drop_duplicates()
    df_pubhistories = df_pubhistories.groupby(['MAGAID','RetractionYear'])['MAGPubYear'].max().reset_index()
    df_pubhistories['YearsActive'] = df_pubhistories['MAGPubYear']-df_pubhistories['RetractionYear']
    df_pubhistories['AttritedClassOld'] = df_pubhistories.apply(lambda row: get_oldattritionclass(row), 
                                                             axis=1)
    df_pubhistories = df_pubhistories.drop(columns=['MAGPubYear','RetractionYear','YearsActive'])
    #df_main = df_main.drop(columns=['AttritedClassOld'])
    df_authors = df_authors.merge(df_pubhistories, on='MAGAID')

In [368]:
dftemp = df_authors[['MAGAID','AttritedClass','AttritedClassOld', 'AttritedClass6']].drop_duplicates()

pd.crosstab(dftemp['AttritedClass6'],dftemp['AttritedClass'])

AttritedClass,-1,0,1
AttritedClass6,,,
-1,1779,1377,50
0,0,10047,0
1,0,1228,1535


In [356]:
df_authors[df_authors['RetractionYear']==2015]['AttritedClass'].value_counts()

AttritedClass
 0    4747
-1     599
 1     467
Name: count, dtype: int64

In [357]:
df_authors[df_authors['RetractionYear']==2012]\
    [['RetractionYear','YearOfAttrition','YearsActive','AttritedClass']].drop_duplicates().head(20)

,RetractionYear,YearOfAttrition,YearsActive,AttritedClass
48,2012.0,-1.0,NaN,0
89,2012.0,2001.0,-11.0,-1
331,2012.0,1997.0,-15.0,-1
981,2012.0,1999.0,-13.0,-1
1045,2012.0,1955.0,-57.0,-1
1782,2012.0,2002.0,-10.0,-1
1935,2012.0,1987.0,-25.0,-1
2126,2012.0,2003.0,-9.0,-1
2678,2012.0,2009.0,-3.0,-1
3260,2012.0,2010.0,-2.0,-1


In [82]:
#df_attrition_year_Bedoor

df_rw = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/retraction_watch/RW_Original_wYear.csv",
                       usecols=['RetractionYear','Record ID'])


histories = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/Retracted_Authors_PubHistories.csv")


notices = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/retraction_notices_postfiltering.csv")






,PID,PaperTitle,OriginalTitle,PubYear,DocSubTypes
0,1591961241,notice of retraction a method of multi dimensi...,Notice of Retraction A method of multi-dimensi...,2010,Retraction Notice
1,1868656167,notice of retraction effect of asparagus polys...,Notice of Retraction Effect of Asparagus polys...,2010,Retraction Notice
2,2042025123,retraction note to therapeutic effects of metf...,Retraction Note to: Therapeutic effects of met...,2016,Retraction Notice
3,2050825760,retraction note to aging decreases rate of doc...,Retraction Note to: Aging decreases rate of do...,2013,Retraction Notice
4,2770316826,notice of retraction a study of the eye catchi...,Notice of Retraction A study of the eye-catchi...,2017,Retraction Notice
...,...,...,...,...,...
15887,2543201822,notice of retraction the development and innov...,Notice of Retraction The development and innov...,2009,Retraction Notice
15888,1588108609,notice of retraction a generalized forward sec...,Notice of Retraction A generalized forward-sec...,2011,Retraction Notice
15889,1513183296,notice of retraction trust performance and inn...,"Notice of Retraction Trust, performance and in...",2011,Retraction Notice
15890,1523723991,notice of retraction a quantitative research o...,Notice of Retraction A quantitative research o...,2011,Retraction Notice


In [83]:
histories = histories[~histories['MAGPID'].isin(notices['PID'])]

In [85]:
histories = histories[histories['MAGPubYear'] <= histories['RetractionYear']]

histories

,MAGPID,MAGAID,MAGAffID,MAGAuthorOrder,MAGTitle,MAGPubYear,MAGAuthorName,Record ID,RetractedPaperMAGPID,RetractionYear
0,524,569306176,148283060.0,1,the cryo thermochromatographic separator cts a...,2002.0,u w kirbach,23311,2273790572,2002.0
1,1598005161,569306176,148283060.0,8,non observation of the production of superheav...,2002.0,u w kirbach,23311,2273790572,2002.0
3,1989365649,569306176,148283060.0,14,chemical investigation of hassium element 108,2002.0,u w kirbach,23311,2273790572,2002.0
5,2018467599,569306176,NaN,9,editorial note observation of superheavy nucle...,2002.0,u w kirbach,23311,2273790572,2002.0
7,2040895841,569306176,148283060.0,8,evidence for new isotopes of element 107 266bh...,2000.0,u w kirbach,23311,2273790572,2002.0
...,...,...,...,...,...,...,...,...,...,...
2841736,3168732224,3170455413,NaN,4,development and application of intensified env...,2014.0,하정민,6759,3168732224,2015.0
2841737,3168732224,3171097566,NaN,3,development and application of intensified env...,2014.0,NaN,6759,3168732224,2015.0
2841738,3168732224,3171812317,NaN,1,development and application of intensified env...,2014.0,youngsu an,6759,3168732224,2015.0
2841739,3168732224,3172678465,NaN,7,development and application of intensified env...,2014.0,noh jungpil,6759,3168732224,2015.0


In [89]:
df_counts = histories.groupby('MAGAID')['MAGPID'].nunique().reset_index().rename(columns={'MAGPID':'numPapers'})

In [90]:
df_counts

,MAGAID,numPapers
0,185294,19
1,224730,11
2,419208,44
3,733318,117
4,861018,59
...,...,...
23615,3177295229,1
23616,3177297137,1
23617,3177442959,1
23618,3177468043,5


In [93]:
df_merged = df_counts.merge(df_attrition_year_Bedoor, on='MAGAID', how='left')

In [96]:
df_merged[~df_merged['YearOfAttrition'].isna()]['numPapers'].describe()

count    19822.000000
mean        57.998083
std        120.704193
min          1.000000
25%          5.000000
50%         17.000000
75%         60.000000
max       5302.000000
Name: numPapers, dtype: float64

In [97]:
# Read PubHistories file
histories = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/Retracted_Authors_PubHistories.csv")
# Read notices file
notices = pd.read_csv("/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/retraction_notices_postfiltering.csv")
# Removing notices from histories
histories = histories[~histories['MAGPID'].isin(notices['PID'])]
# Removing all papers of retracted authors that are post retraction year
histories = histories[histories['MAGPubYear'] <= histories['RetractionYear']]
# Counting number of papers of retracted authors by the time of retraction
df_counts = histories.groupby('MAGAID')['MAGPID'].nunique().reset_index().rename(columns={'MAGPID':'numPapers'})

# Reading Bedoor's file on attrition year
df_attrition_year_Bedoor = pd.read_csv(OUTDIR+"AIDs_YearOfAttrition_95percentile.csv")\
                            .rename(columns={'AID':'MAGAID'})

# Merging Bedoor's file with counts (merge left to retain all authors whether they are in Bedoor's file or not)
df_merged = df_counts.merge(df_attrition_year_Bedoor, on='MAGAID', how='left')

df_merged[df_merged['YearOfAttrition'].isna()]['numPapers'].describe()



count    3798.000000
mean        1.003949
std         0.062729
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: numPapers, dtype: float64

In [99]:
df_merged[df_merged['YearOfAttrition'].isna() & df_merged['numPapers'].gt(1)]

,MAGAID,numPapers,YearOfAttrition
1033,372365273,2,NaN
15322,2484907224,2,NaN
15524,2494630675,2,NaN
16332,2558346274,2,NaN
16360,2559825382,2,NaN
16769,2578912753,2,NaN
17912,2631961372,2,NaN
17991,2633674136,2,NaN
18254,2641294629,2,NaN
18586,2651016970,2,NaN


In [102]:
df_merged[~df_merged['YearOfAttrition'].isna() & df_merged['numPapers'].eq(1)]

,MAGAID,numPapers,YearOfAttrition
68,24132859,1,-1.0
130,42486839,1,2011.0
293,109484254,1,2015.0
362,137284131,1,2009.0
491,188140926,1,-1.0
...,...,...,...
23524,3171743004,1,-1.0
23530,3172090888,1,2003.0
23544,3173003562,1,2015.0
23551,3173337439,1,2014.0
